In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

In [ ]:
df = pd.read_csv('Resources/98-401-X2021002_English_CSV_data.csv', encoding='ISO-8859-1')

In [ ]:
#reducing columns
columns_to_keep=['GEO_NAME','CHARACTERISTIC_ID','CHARACTERISTIC_NAME','C1_COUNT_TOTAL']
reduced_df=df[columns_to_keep]

In [ ]:
#grabbing just the population for each city
pop_df = reduced_df[reduced_df['CHARACTERISTIC_ID'] == 1]
pop_group=pop_df.groupby(['GEO_NAME','CHARACTERISTIC_NAME'])
pop_group['C1_COUNT_TOTAL'].max().sort_values(ascending=False).head(15)

In [ ]:
#filtering by top 15 cities by population
top_15_cities = pop_group['C1_COUNT_TOTAL'].max().sort_values(ascending=False).head(15)
top_15_cities = top_15_cities.index.get_level_values(0)
mask = reduced_df['GEO_NAME'].isin(top_15_cities)
top_15_pop_df = reduced_df[mask]

In [ ]:
#get the names of the top 15 cities
cities = top_15_pop_df['GEO_NAME'].unique()
income_data = []

In [ ]:
#get city name and median income for each city
for city in cities:

    city_row = top_15_pop_df[(top_15_pop_df['GEO_NAME']==city) & (top_15_pop_df['CHARACTERISTIC_ID'] == 119)]
    city_name = city_row['GEO_NAME'].iloc[0]
    median_income = city_row['C1_COUNT_TOTAL'].iloc[0]

    income_data.append({
        "City":city_name,
        "Median Income":median_income
    })

income_df = pd.DataFrame(income_data)    

In [ ]:
income_df

In [ ]:
weather_df = pd.read_csv('Output/weather_data.csv')
columns_to_keep=['City','Max Temp','Wind Speed']
weather_df= weather_df[columns_to_keep]
#weather_df=weather_df.rename(columns={'GEO':'City'})
weather_df.head(15)

In [ ]:
merged_df = pd.merge(income_df,weather_df,on='City',how='inner')
merge_to_keep = ['City','Median Income','Max Temp','Wind Speed']
merged_df = merged_df[merge_to_keep]

In [ ]:
merged_data=pd.read_csv('Output/merged_data.csv')

In [ ]:
merged_data

## Temperature vs Median Income

In [ ]:
def linear_regression(x,y, tickinterval=None):
    
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
    regress_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

    plt.scatter(x,y)
    plt.plot(x,regress_values,"r-")
    plt.annotate(line_eq,(x.min(), regress_values.max()),fontsize=15,color="red")
    print(f'The r-value is {rvalue}')

    #plt.xlim(0.9 * x.min(), 1.1 * x.max())
    
    if tickinterval is not None:
        plt.xticks(np.arange(x.min(), x.max()+tickinterval, tickinterval))

In [ ]:
y_values = merged_data['Median Income']
x_values = merged_data['Max Temp']
linear_regression(x_values,y_values)
plt.xlabel('Max Temp(C)')
plt.ylabel('Median Income($)')
plt.title('Top 15: Max Temp vs Median Income')
plt.savefig('Output/top15_MaxTempvsMedianIncome.png')

## Analysis
There is a weak negative correlation between max temperature and median income in the top 15 cities.  As the temperature increases the median income decreases.  However, there are other factors that would influence this value. 

In [ ]:
#filter for the value you want and sort so they're in order.
median_income_df = reduced_df[reduced_df['CHARACTERISTIC_ID'] == 119][['GEO_NAME', 'C1_COUNT_TOTAL']].sort_values(by='C1_COUNT_TOTAL',ascending=False)
city_name=median_income_df['GEO_NAME'].unique()
income_data2=[]

In [ ]:
median_income_df

In [ ]:
#add a median income rank
i=1
for city in city_name:
    
    city_row = median_income_df[median_income_df['GEO_NAME']==city]
    city_name = city_row['GEO_NAME'].iloc[0]
    median_income = city_row['C1_COUNT_TOTAL'].iloc[0]
    income_rank = i

    income_data2.append({
        "City":city_name,
        "Median Income":median_income,
        "Rank": income_rank
    })
    i+=1
income_df2=pd.DataFrame(income_data2)

In [ ]:
income_df2

In [ ]:
#rename to city so we can compare to the top_15_cities
income_df2 = income_df2.rename(columns={'City':'GEO_NAME'})

In [ ]:
#filter for top 15 cities
mask = income_df2['GEO_NAME'].isin(top_15_cities)
top_15_income = income_df2[mask]

In [ ]:
#export to csv
top_15_income.to_csv('Output/income_data.csv',index=False)

In [ ]:
#rename so we can merge on 'City'
income_df2 = income_df2.rename(columns={'GEO_NAME':'City'})

In [ ]:
weather_df2= pd.read_csv('Output/weather_data_larger_set.csv')
merged_df2 = pd.merge(income_df2,weather_df2,on='City',how='inner')

In [ ]:
x_values = merged_df2['Max Temp']
y_values = merged_df2['Median Income']
linear_regression(x_values,y_values)
plt.xlabel("Temp(C)")
plt.ylabel("Median Income($)")
plt.title('All Cities: Max Temp vs Median Income')
plt.savefig('Output/AllCities_MaxTempvsMedianIncome.png')


## Analysis
There is a stronger negative correlation when looking at all of the cities in Canada.  However, it is still not strong enough to draw any conclusions from

In [ ]:
merged_df2.groupby(['City','Max Temp'])['Median Income'].max().sort_values(ascending=False).tail(25)

In [ ]:
pop = reduced_df[reduced_df['CHARACTERISTIC_ID'] == 1]['C1_COUNT_TOTAL']
income=income_df2['Median Income']

## Population vs Median Income

In [ ]:
# having trouble getting this to look nice on a graph. Also not 100% sure I understand non-linear regression

In [ ]:
#y_values=income
#x_values=pop
#linear_regression(x_values,y_values,tickinterval=1000000)

In [ ]:
def non_linear_regression(x, y, degree=2):
    x = np.array(x)
    y = np.array(y)
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(x.reshape(-1, 1))
    model = LinearRegression()
    model.fit(X_poly, y)
    return model, poly
    

In [ ]:
x_values_np = x_values.to_numpy()
model, poly = non_linear_regression(x_values, y_values, degree=2)
predictions = model.predict(poly.fit_transform(x_values_np.reshape(-1, 1)))

In [ ]:
plt.scatter(x_values, y_values)
plt.plot(x_values, predictions, color='red')
plt.xscale("log")
plt.xlabel("Population")
plt.ylabel("Median Income")
plt.show()